In [1]:
# %% 1. การติดตั้งและ Import Library ที่จำเป็น

# ติดตั้ง library ที่จำเป็น, โดยเฉพาะ accelerate เพื่อการโหลดโมเดลขนาดใหญ่
!pip install -q transformers torch datasets accelerate

import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm.auto import tqdm

# ตรวจสอบว่ามี GPU ให้ใช้หรือไม่
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")
print(f"Pytorch version: {torch.__version__}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.0 MB/s eta 0:00:00
อุปกรณ์ที่ใช้: cuda
Pytorch version: 2.6.0+cu124


In [14]:
df

,review_text,aspect,sentiment
0,บ้านน่านเป็นโรงแรมขนาดไม่ใหญ่ มีห้องอาหารอยู่ช...,รสชาติ,positive
1,ลุงสนั่นหมูกระทะเป็นการขายแบบเป็นชุดๆ\nโดยเริ่...,บรรยากาศ,positive
2,เคยมาเที่ยวไหว้พระธาตุไชยาเมื่อปีที่แล้วตอนสงก...,บริการ,positive
3,บรรยากาศร้านสวย น่านั่งชิวๆ เดินทางสะดวก\nราคา...,บรรยากาศ,positive
4,ใครที่ชอบอาหารจีนแนวภัตตาคารละก็ห้ามพลาดร้านนี...,ราคา,positive
...,...,...,...
85552,ลักษณะร้าน + บรรยากาศ \nร้าน มงคลชัย เป็นร้านข...,ราคา,positive
85553,ตอนแรกเข้ามาทานแบบไม่คาดหวัง แต่พอได้ทานพิซซ่า...,ราคา,positive
85554,เป็นร้านที่บรรยากาศดีอีกร้านหนึ่งของจังหวัดตรา...,ราคา,positive
85555,ร้านนี้หาไม่ยากครับ จากถนนเพชรเกษม เลี้ยวเข้าต...,บริการ,positive


In [15]:
# %% การเตรียมข้อมูล
print("กำลังโหลด Wongnai Reviews Dataset...")
# โหลดชุดข้อมูลจาก Hugging Face

df = pd.read_csv('/content/train_data.csv')

# สร้างข้อมูล test โดยการสุ่มมา 500 ตัวอย่าง (ในการทดลอง อาจลดเหลือ 10-20)
test_df = df.sample(n=500, random_state=42).reset_index(drop=True)
test_df['review_id'] = test_df.index
print("\nสร้าง Test Set สำหรับทดลอง Pipeline สำเร็จ")
print("ตัวอย่างข้อมูล Test Set 5 แถวแรก:")
print(test_df.head())

กำลังโหลด Wongnai Reviews Dataset...

สร้าง Test Set สำหรับทดลอง Pipeline สำเร็จ
ตัวอย่างข้อมูล Test Set 5 แถวแรก:
                                         review_text    aspect sentiment  \
0  มาทานเนื่องจากร้านนี้อยู่ใกล้บ้าน และบรรยากาศท...  บรรยากาศ  positive   
1  วันนี้มาธุระแถวตลิ่งชัน เลยแวะเติมพลังหาอะไรทา...    รสชาติ  positive   
2  ขนมอาโป๊ง ขนมชื่อไม่คุ้นหู สำหรับหลายๆคน เพราะ...  บรรยากาศ  positive   
3  ร้าน La Montée Seoul เป็นร้านเบเกอรี่สไตล์ฝรั่...      ราคา  positive   
4  วันนี้มีโอกาศผ่านมาทานร้านนี้ครับ ร้านตั้งอยู่...      ราคา  positive   

   review_id  
0          0  
1          1  
2          2  
3          3  
4          4  


In [17]:
test_df

,review_text,aspect,sentiment,review_id
0,มาทานเนื่องจากร้านนี้อยู่ใกล้บ้าน และบรรยากาศท...,บรรยากาศ,positive,0
1,วันนี้มาธุระแถวตลิ่งชัน เลยแวะเติมพลังหาอะไรทา...,รสชาติ,positive,1
2,ขนมอาโป๊ง ขนมชื่อไม่คุ้นหู สำหรับหลายๆคน เพราะ...,บรรยากาศ,positive,2
3,ร้าน La Montée Seoul เป็นร้านเบเกอรี่สไตล์ฝรั่...,ราคา,positive,3
4,วันนี้มีโอกาศผ่านมาทานร้านนี้ครับ ร้านตั้งอยู่...,ราคา,positive,4
...,...,...,...,...
495,ปกติเคยทานแต่บุฟเฟ่ต์ของร้านนี้ ไม่เคยสังเกตุว...,ราคา,positive,495
496,มีนัด central world เพื่อนรอ coffee world ชั้น...,รสชาติ,positive,496
497,หลังจากของคาวแล้ว เติมของหวานต่อทันที \nร้านนี...,รสชาติ,positive,497
498,690 net/คน ชาบู+ชูชิ(จะแบ่งเป็น2ชุดAกับBวันที่...,บริการ,positive,498


In [20]:
# %% การเลือกรุ่นโมเดล (Model Selection)

# --- เลือกโมเดลที่นี่ ---
# model_id = "Qwen/Qwen2-7B-Instruct"
model_id = "scb10x/typhoon2-qwen2.5-7b-instruct"

print(f"โมเดลที่เลือกคือ: {model_id}")


โมเดลที่เลือกคือ: scb10x/typhoon2-qwen2.5-7b-instruct


In [21]:
# %% 4. การโหลดโมเดลและ Tokenizer

print(f"กำลังโหลด Tokenizer จาก '{model_id}'...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f"กำลังโหลดโมเดล '{model_id}'...")
print("หมายเหตุ: ขั้นตอนนี้อาจใช้เวลาหลายนาทีและต้องการ VRAM ประมาณ 14-16 GB")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # ใช้ bfloat16 เพื่อประสิทธิภาพและความเร็ว
    device_map="auto" # สั่งให้ transformers จัดการการวางโมเดลบน GPU อัตโนมัติ
)

print("\nโหลดโมเดลและ Tokenizer สำเร็จ!")


กำลังโหลด Tokenizer จาก 'scb10x/typhoon2-qwen2.5-7b-instruct'...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]


โหลดโมเดลและ Tokenizer สำเร็จ!


In [22]:
# %% การออกแบบ Prompt Template (Few-Shot)

# สร้างโครงสร้าง Prompt แบบ Chat
# ประกอบด้วย:
# 1. System: บอกบทบาทและหน้าที่ของ LLM
# 2. User/Assistant (Few-shot): ตัวอย่างคำถาม-คำตอบที่ดี 2-3 ชุด
# 3. User (Actual task): คำถามจริงที่เราต้องการคำตอบ

# เราจะเก็บ template นี้ไว้ในรูปแบบ list ของ dictionaries
# โดยจะเติมรีวิวจริงเข้าไปในภายหลัง
chat_template = [
    {
        "role": "system",
        "content": "คุณคือผู้เชี่ยวชาญด้านการวิเคราะห์รีวิวร้านอาหาร หน้าที่ของคุณคือสกัด 'แง่มุม' (aspect) และ 'ความรู้สึก' (sentiment) ที่เกี่ยวข้องจากข้อความรีวิว และตอบกลับในรูปแบบ JSON Array ที่กำหนดเท่านั้น โดยไม่ต้องมีข้อความอธิบายใดๆ เพิ่มเติม"
    },
    {
        "role": "user",
        "content": """รีวิว: "อาหารอร่อยมาก แต่ราคาแอบแรงไปนิดนึง บรรยากาศร้านดีครับ" จงสกัดแง่มุมและความรู้สึกออกมา"""
    },
    {
        "role": "assistant",
        "content": """[{"aspect": "รสชาติ", "sentiment": "positive"}, {"aspect": "ราคา", "sentiment": "negative"}, {"aspect": "บรรยากาศ", "sentiment": "positive"}]"""
    },
    {
        "role": "user",
        "content": """รีวิว: "โดยรวมโอเคเลยนะ รสชาติใช้ได้ แต่พนักงานบริการช้าไปหน่อย ต้องเรียกหลายรอบ" จงสกัดแง่มุมและความรู้สึกออกมา"""
    },
    {
        "role": "assistant",
        "content": """[{"aspect": "รสชาติ", "sentiment": "neutral"}, {"aspect": "บริการ", "sentiment": "negative"}]"""
    },
    {
        "role": "user",
        "content": """รีวิว: "{review_text}" จงสกัดแง่มุมและความรู้สึกออกมา"""
    }
]

print("สร้าง Chat Template สำเร็จแล้ว")

# --- ทดลองสร้าง Prompt สำหรับรีวิวแรกใน Test set ---
first_review = test_df['review_text'].iloc[0]
prompt_for_first_review = tokenizer.apply_chat_template(
    [
        *chat_template[:-1], # เอา template ทั้งหมดยกเว้น user prompt สุดท้าย
        {"role": "user", "content": chat_template[-1]["content"].format(review_text=first_review)}
    ],
    tokenize=False,
    add_generation_prompt=True
)

print("\n--- ตัวอย่าง Prompt ที่จะส่งให้โมเดล ---")
print(prompt_for_first_review)


สร้าง Chat Template สำเร็จแล้ว

--- ตัวอย่าง Prompt ที่จะส่งให้โมเดล ---
<|im_start|>system
คุณคือผู้เชี่ยวชาญด้านการวิเคราะห์รีวิวร้านอาหาร หน้าที่ของคุณคือสกัด 'แง่มุม' (aspect) และ 'ความรู้สึก' (sentiment) ที่เกี่ยวข้องจากข้อความรีวิว และตอบกลับในรูปแบบ JSON Array ที่กำหนดเท่านั้น โดยไม่ต้องมีข้อความอธิบายใดๆ เพิ่มเติม<|im_end|>
<|im_start|>user
รีวิว: "อาหารอร่อยมาก แต่ราคาแอบแรงไปนิดนึง บรรยากาศร้านดีครับ" จงสกัดแง่มุมและความรู้สึกออกมา<|im_end|>
<|im_start|>assistant
[{"aspect": "รสชาติ", "sentiment": "positive"}, {"aspect": "ราคา", "sentiment": "negative"}, {"aspect": "บรรยากาศ", "sentiment": "positive"}]<|im_end|>
<|im_start|>user
รีวิว: "โดยรวมโอเคเลยนะ รสชาติใช้ได้ แต่พนักงานบริการช้าไปหน่อย ต้องเรียกหลายรอบ" จงสกัดแง่มุมและความรู้สึกออกมา<|im_end|>
<|im_start|>assistant
[{"aspect": "รสชาติ", "sentiment": "neutral"}, {"aspect": "บริการ", "sentiment": "negative"}]<|im_end|>
<|im_start|>user
รีวิว: "มาทานเนื่องจากร้านนี้อยู่ใกล้บ้าน และบรรยากาศที่ติดริมแม่น้ำ ภายในร้านจะมีส่วนท

In [23]:
# %% การเริ่ม Pipeline การทำนายผล

results = []
# เพื่อความรวดเร็วในการทดลอง เราจะทำแค่ 20 รีวิวแรก
# ในการแข่งจริง ให้ใช้ test_df ทั้งหมด
test_subset = test_df.head(20)

print(f"กำลังเริ่มทำนายผลสำหรับรีวิว {len(test_subset)} ตัวอย่าง...")

for index, row in tqdm(test_subset.iterrows(), total=len(test_subset)):
    review_id = row['review_id']
    review_text = row['review_text']

    # สร้าง Prompt สำหรับรีวิวปัจจุบัน
    current_chat = [
        *chat_template[:-1],
        {"role": "user", "content": chat_template[-1]["content"].format(review_text=review_text)}
    ]

    # แปลง prompt เป็น token id
    inputs = tokenizer.apply_chat_template(
        current_chat,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(DEVICE)

    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=256,
            do_sample=False, # ใช้ do_sample=False เพื่อให้ได้คำตอบที่แน่นอน
            pad_token_id=tokenizer.eos_token_id
        )

    # ถอดรหัสเฉพาะส่วนที่ถูกสร้างขึ้นใหม่
    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # --- ส่วนของการ Parse ผลลัพธ์ ---
    try:
        # ค้นหาเฉพาะส่วนที่เป็น JSON [ ... ]
        json_part = response_text[response_text.find('['):response_text.rfind(']')+1]
        parsed_output = json.loads(json_part)

        # ตรวจสอบว่าเป็น list หรือไม่
        if isinstance(parsed_output, list):
            for item in parsed_output:
                # ตรวจสอบว่า item เป็น dict และมี key ครบถ้วน
                if isinstance(item, dict) and 'aspect' in item and 'sentiment' in item:
                    results.append({
                        'id': review_id,
                        'aspect': item['aspect'],
                        'sentiment': item['sentiment']
                    })
    except (json.JSONDecodeError, IndexError) as e:
        print(f"\nไม่สามารถ Parse ผลลัพธ์สำหรับ review_id {review_id}: {e}")
        print(f"  -> ผลลัพธ์จากโมเดล: {response_text}")

print("\nการทำนายผลเสร็จสิ้น")


กำลังเริ่มทำนายผลสำหรับรีวิว 20 ตัวอย่าง...


  0%|          | 0/20 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'


การทำนายผลเสร็จสิ้น


In [24]:
# %% การสร้างไฟล์ Submission

print("กำลังสร้างไฟล์ submission.csv...")

if not results:
    print("ไม่พบผลลัพธ์ที่ถูกต้อง! ไม่สามารถสร้างไฟล์ submission ได้")
else:
    # สร้าง DataFrame จาก list ของผลลัพธ์
    submission_df = pd.DataFrame(results)

    # จัดลำดับคอลัมน์ให้ถูกต้อง
    submission_df = submission_df[['id', 'aspect', 'sentiment']]

    print("\nตัวอย่างผลลัพธ์ 10 แถวแรก:")
    print(submission_df.head(10))

    # บันทึกเป็นไฟล์ CSV
    submission_df.to_csv("submission.csv", index=False)
    print("\nสร้างไฟล์ submission.csv สำเร็จ!")


กำลังสร้างไฟล์ submission.csv...

ตัวอย่างผลลัพธ์ 10 แถวแรก:
   id             aspect sentiment
0   0           บรรยากาศ  positive
1   0             บริการ  positive
2   0             รสชาติ  positive
3   0  ความสดของวัตถุดิบ  negative
4   0             ปริมาณ  negative
5   1           บรรยากาศ  positive
6   1             บริการ  positive
7   1             รสชาติ  positive
8   1               ราคา   neutral
9   2             รสชาติ   neutral

สร้างไฟล์ submission.csv สำเร็จ!
